# TFID TECHNIQUE

In [3]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.model_selection import GridSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC

# # Load your dataset
# dataset = pd.read_csv("Training_Essay_Data.csv")

# # Split the dataset
# X_train, X_test, y_train, y_test = train_test_split(
#     dataset['text'], dataset['generated'].astype(int), test_size=0.2, random_state=42
# )

# # Create a pipeline with TfidfVectorizer and Multinomial Naive Bayes
# text_clf = Pipeline([
#     ('tfidf', TfidfVectorizer()),
#     ('clf', MultinomialNB())
# ])

# # Define parameters for Grid Search
# param_grid = {
#     'tfidf__ngram_range': [(1, 1), (1, 2)],
#     'clf__alpha': [0.5, 1.0],
# }

# # Perform Grid Search for hyperparameter tuning
# grid_search = GridSearchCV(text_clf, param_grid, cv=5)
# grid_search.fit(X_train, y_train)

# # Get the best model from Grid Search
# best_model = grid_search.best_estimator_

# # Evaluate the model on the test set
# accuracy = best_model.score(X_test, y_test)
# print(f"Accuracy: {accuracy:.2f}")


# Function to prompt a choice to input text manually or via .doc or .txt (File Path)

In [4]:
# def classify_text(input_text, model):
#     # Predict the class
#     prediction = model.predict([input_text])

#     # Output the result
#     if prediction[0] == 0:
#         return "Human-generated"
#     else:
#         return "AI-generated"

# # Function to read text from a file
# def read_text_from_file(file_path):
#     try:
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     except Exception as e:
#         print(f"Error reading file: {e}")
#         return None

# # Prompt user to choose an option
# print("Choose an option:")
# print("1. Enter multiline text manually")
# print("2. Upload a doc or text file")

# choice = input("Enter your choice (1 or 2): ")

# if choice == '1':
#     user_input_lines = []
#     print("Enter multiline text. Press Enter twice to finish.")
#     while True:
#         line = input()
#         if not line:
#             break
#         user_input_lines.append(line)
#     user_input = '\n'.join(user_input_lines)
    
#     result = classify_text(user_input, best_model)

# elif choice == '2':
#     # Prompt user to enter the file path
#     file_path = input("Enter the path of the text or doc file: ")

#     # Check if the file is a doc file and read its content
#     if file_path.endswith('.docx'):
#         try:
#             doc = Document(file_path)
#             doc_text = ' '.join([paragraph.text for paragraph in doc.paragraphs])
#             result = classify_text(doc_text, best_model)
#         except Exception as e:
#             print(f"Error reading doc file: {e}")
#             result = None
#     else:
#         # Read text from the file
#         input_text = read_text_from_file(file_path)

#         # Classify the text
#         if input_text:
#             result = classify_text(input_text, best_model)
#         else:
#             result = None
# else:
#     print("Invalid choice. Please enter 1 or 2.")

# # Display the result
# if result is not None:
#     print(f"The input text is classified as: {result}")
# else:
#     print("Classification failed.")

# GloVe EMBEDDING TECHNIQUE

In [5]:
import numpy as np

def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype="float32")
            embeddings_index[word] = coefs
    return embeddings_index

glove_file_path = "glove.6B.100d.txt"  # Adjust the path based on your downloaded file
glove_embeddings = load_glove_embeddings(glove_file_path)


# TOKENIZE TEXT

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd

dataset = pd.read_csv("train_essays.csv")

X_train, X_test, y_train, y_test = train_test_split(
    dataset['text'], dataset['generated'].astype(int), test_size=0.2, random_state=42
)

max_words = 10000  # Choose an appropriate value
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)


# PAD SEQUENCES

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = 100  # Choose an appropriate value
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)


# CREATE EMBEDDING MATRIX

In [8]:
word_index = tokenizer.word_index
embedding_dim = 100  # Use the same dimension as your GloVe file
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = glove_embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# BUILD THE MODEL

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model.add(Flatten())
model.add(Dense(1, activation="sigmoid"))


# Add BatchNormalization after Dense layers
model.add(BatchNormalization())

# Compile the model (you may need to adjust the loss function and metrics based on your problem)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test))


Epoch 1/10


2024-01-21 12:58:03.688944: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


35/35 [==============================] - 0s 3ms/step - loss: 0.8243 - accuracy: 0.9165 - val_loss: 0.0559 - val_accuracy: 0.9964
Epoch 2/10
35/35 [==============================] - 0s 1ms/step - loss: 0.0280 - accuracy: 0.9982 - val_loss: 0.0559 - val_accuracy: 0.9964
Epoch 3/10
35/35 [==============================] - 0s 925us/step - loss: 0.0280 - accuracy: 0.9982 - val_loss: 0.0559 - val_accuracy: 0.9964
Epoch 4/10
35/35 [==============================] - 0s 925us/step - loss: 0.0280 - accuracy: 0.9982 - val_loss: 0.0559 - val_accuracy: 0.9964
Epoch 5/10
35/35 [==============================] - 0s 910us/step - loss: 0.0280 - accuracy: 0.9982 - val_loss: 0.0559 - val_accuracy: 0.9964
Epoch 6/10
35/35 [==============================] - 0s 904us/step - loss: 0.0280 - accuracy: 0.9982 - val_loss: 0.0559 - val_accuracy: 0.9964
Epoch 7/10
35/35 [==============================] - 0s 967us/step - loss: 0.0280 - accuracy: 0.9982 - val_loss: 0.0559 - val_accuracy: 0.9964
Epoch 8/10
35/35 [===

In [10]:
dataset.shape

(1378, 4)

In [11]:
import pandas as pd

# Assuming 'label' is the name of the label column in your DataFrame
# Replace 'your_dataframe.csv' with the actual file path or DataFrame variable
dataset = pd.read_csv("train_essays.csv")

# Count occurrences of 1 and 0 in the 'label' column
label_counts = dataset['generated'].value_counts()

# Print the counts
print("Count of 1:", label_counts.get(1, 0))  # 0 if there are no occurrences of 1
print("Count of 0:", label_counts.get(0, 0))  # 0 if there are no occurrences of 0


Count of 1: 3
Count of 0: 1375


# Function to prompt a choice to input text manually or via .doc or .txt (File Path)

In [12]:
# def classify_text(input_text, model, tokenizer, max_sequence_length):
#     # Tokenize and pad the input text
#     input_sequence = tokenizer.texts_to_sequences([input_text])
#     input_padded = pad_sequences(input_sequence, maxlen=max_sequence_length)

#     # Predict the class
#     prediction = model.predict(input_padded)

#     # Output the result
#     if prediction[0] < 0.5:
#         return "Human-generated"
#     else:
#         return "AI-generated"

# # Function to read text from a file
# def read_text_from_file(file_path):
#     try:
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     except Exception as e:
#         print(f"Error reading file: {e}")
#         return None

# # Prompt user to choose an option
# print("Choose an option:")
# print("1. Enter multiline text manually")
# print("2. Upload a doc or text file")

# choice = input("Enter your choice (1 or 2): ")

# if choice == '1':
#     user_input_lines = []
#     print("Enter multiline text. Press Enter twice to finish.")
#     while True:
#         line = input()
#         if not line:
#             break
#         user_input_lines.append(line)
#     user_input = '\n'.join(user_input_lines)
    
#     result = classify_text(user_input, model= model, tokenizer=tokenizer, max_sequence_length=max_sequence_length)

# elif choice == '2':
#     # Prompt user to enter the file path
#     file_path = input("Enter the path of the text or doc file: ")

#     # Check if the file is a doc file and read its content
#     if file_path.endswith('.docx'):
#         try:
#             doc = Document(file_path)
#             doc_text = ' '.join([paragraph.text for paragraph in doc.paragraphs])
#             result = classify_text(doc_text, model= model, tokenizer=tokenizer, max_sequence_length=max_sequence_length)
#         except Exception as e:
#             print(f"Error reading doc file: {e}")
#             result = None
#     else:
#         # Read text from the file
#         input_text = read_text_from_file(file_path)

#         # Classify the text
#         if input_text:
#             result = classify_text(input_text, model= model, tokenizer=tokenizer, max_sequence_length=max_sequence_length)
#         else:
#             result = None
# else:
#     print("Invalid choice. Please enter 1 or 2.")

# # Display the result
# if result is not None:
#     print(f"The input text is classified as: {result}")
# else:
#     print("Classification failed.")


In [13]:
# import tkinter as tk
# from tkinter import scrolledtext, messagebox, filedialog
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from docx import Document

# def classify_text(input_text, model, tokenizer, max_sequence_length):
#     # Tokenize and pad the input text
#     input_sequence = tokenizer.texts_to_sequences([input_text])
#     input_padded = pad_sequences(input_sequence, maxlen=max_sequence_length)

#     # Predict the class
#     prediction = model.predict(input_padded)

#     # Output the result
#     if prediction[0] < 0.5:
#         return "Human-generated"
#     else:
#         return "AI-generated"

# def read_text_from_file(file_path):
#     try:
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     except Exception as e:
#         print(f"Error reading file: {e}")
#         return None

# def calculate_classification():
#     user_input = text_entry.get("1.0", "end-1c")
#     if user_input.strip() == "":
#         messagebox.showinfo("Error", "Please enter text.")
#         return

#     result = classify_text(user_input, model=model, tokenizer=tokenizer, max_sequence_length=max_sequence_length)
#     result_label.config(text=f"The input text is classified as: {result}")

# def open_file_dialog():
#     file_path = filedialog.askopenfilename(title="Select a file", filetypes=[("Text files", "*.txt"), ("Word files", "*.docx")])
#     if file_path:
#         if file_path.endswith('.docx'):
#             try:
#                 doc = Document(file_path)
#                 doc_text = ' '.join([paragraph.text for paragraph in doc.paragraphs])
#                 text_entry.delete("1.0", "end")
#                 text_entry.insert("1.0", doc_text)
#             except Exception as e:
#                 messagebox.showinfo("Error", f"Error reading doc file: {e}")
#         else:
#             input_text = read_text_from_file(file_path)
#             if input_text:
#                 text_entry.delete("1.0", "end")
#                 text_entry.insert("1.0", input_text)
#             else:
#                 messagebox.showinfo("Error", "Error reading file.")

# # Create the main window
# window = tk.Tk()
# window.title("Text Classification GUI")

# # Create text entry
# text_entry = scrolledtext.ScrolledText(window, width=40, height=10, wrap=tk.WORD)
# text_entry.pack(padx=10, pady=10)

# # Create buttons
# calculate_button = tk.Button(window, text="Calculate", command=calculate_classification)
# calculate_button.pack(pady=5)

# open_file_button = tk.Button(window, text="Open File", command=open_file_dialog)
# open_file_button.pack(pady=5)

# # Create result label
# result_label = tk.Label(window, text="")
# result_label.pack(pady=5)

# # Run the GUI
# window.mainloop()


In [14]:
import sys
import pickle
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QPushButton, QTextEdit, QFileDialog, QHBoxLayout
from tensorflow.keras.preprocessing.sequence import pad_sequences



def classify_text(input_text, model, tokenizer, max_sequence_length):
    # Tokenize and pad the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_padded = pad_sequences(input_sequence, maxlen=max_sequence_length)

    # Predict the class
    prediction = model.predict(input_padded)

    # Output the result
    if prediction[0] < 0.5:
        return "Human-generated"
    else:
        return "AI-generated"

class TextClassificationApp(QWidget):
    def __init__(self, model, tokenizer, max_sequence_length):
        super().__init__()

        self.model = model
        self.tokenizer = tokenizer
        self.max_sequence_length = max_sequence_length

        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout()

        # Text editor
        self.text_edit = QTextEdit()
        layout.addWidget(self.text_edit)

        # Buttons layout
        buttons_layout = QHBoxLayout()

        # Calculate button
        calculate_button = QPushButton('Calculate')
        calculate_button.clicked.connect(self.calculate_button_clicked)
        buttons_layout.addWidget(calculate_button)

        # Upload button
        upload_button = QPushButton('Upload File')
        upload_button.clicked.connect(self.upload_button_clicked)
        buttons_layout.addWidget(upload_button)

        # Close button
        close_button = QPushButton('Close')
        close_button.clicked.connect(self.close_application)
        buttons_layout.addWidget(close_button)

        layout.addLayout(buttons_layout)

        # Result label
        self.result_label = QLabel('')
        layout.addWidget(self.result_label)

        self.setLayout(layout)
        self.setWindowTitle('Text Classification App')
        self.setGeometry(100, 100, 600, 400)

    def calculate_button_clicked(self):
        input_text = self.text_edit.toPlainText()
        if input_text:
            result = classify_text(input_text, self.model, self.tokenizer, self.max_sequence_length)
            self.result_label.setText(f'The input text is classified as: {result}')
        else:
            self.result_label.setText('Please enter text.')

    def upload_button_clicked(self):
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, 'Upload Text File', '', 'Text Files (*.txt);;All Files (*)')
        if file_path:
            with open(file_path, 'r', encoding='utf-8') as file:
                file_content = file.read()
                self.text_edit.setPlainText(file_content)
            self.result_label.clear()

    def close_application(self):
        # Explicitly release resources or perform cleanup if needed
        self.close()

# Rest of your code remains unchanged

if __name__ == '__main__':
    # Replace 'model', 'tokenizer', and 'max_sequence_length' with your actual model and tokenizer
    model = model
    tokenizer = tokenizer
    max_sequence_length = 100  # Replace with your actual value

    app = QApplication(sys.argv)
    window = TextClassificationApp(model, tokenizer, max_sequence_length)
    window.show()
    sys.exit(app.exec_())


SystemExit: 0

/Users/saksham2510/anaconda3/envs/nlp_env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
